In [33]:
!pip install python_speech_features

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import zipfile
pickle ='./drive/MyDrive/bestpickles.zip'
zip_ref=zipfile.ZipFile(pickle,'r')
zip_ref.extractall('./')
zip_ref.close()

In [36]:
models = './drive/MyDrive/bestmodels.zip'
zip_ref=zipfile.ZipFile(models,'r')
zip_ref.extractall('./')
zip_ref.close()

In [37]:
test = './drive/MyDrive/test.zip'
zip_ref=zipfile.ZipFile(test,'r')
zip_ref.extractall('./')
zip_ref.close()

In [38]:
import pickle
import os
import numpy as np
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc
from keras.models import load_model
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [39]:
def build_predictions(audio_dir):
    y_true = []
    y_pred = []
    fn_prob = {}

    print('Extracting features from audio')
    for fn in tqdm(os.listdir(audio_dir)):
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        label = fn2class[fn]
        c = classes.index(label)
        y_prob = []

        for i in range(0, wav.shape[0]-config.step, config.step):
            sample = wav[i:i+config.step]
            x = mfcc(sample, rate, numcep=config.nfeat,
                     nfilt=config.nfilt, nfft=config.nfft).T if config.mode =='conv' else mfcc(sample, rate, numcep=config.nfeat,
                     nfilt=config.nfilt, nfft=config.nfft)
            x = (x-config.min)/(config.max-config.min)

            if config.mode == 'conv':
                x = x.reshape(1, x.shape[0], x.shape[1], 1)
            elif config.mode == 'time':
                x = np.expand_dims(x, axis=0)
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)

        fn_prob[fn] = np.mean(y_prob, axis=0).flatten()

    return y_true, y_pred, fn_prob


In [40]:
df = pd.read_csv('./drive/MyDrive/test_mapping.csv')
classes = list(np.unique(df.label))
fn2class = dict(zip(df.fname, df.label))
p_path = os.path.join('pickles', 'time.p')


In [41]:
class Config:
    def __init__(self, mode='time', nfilt=26, nfeat=13, nfft=512, rate=16000):
        self.mode=mode
        self.nfilt=nfilt
        self.nfeat=nfeat
        self.nfft = nfft
        self.rate=rate
        self.step=int(rate/10)
        self.model_path = os.path.join('models', mode + '.model')
        self.p_path = os.path.join('pickles', mode + '.p')

with open(p_path, 'rb') as handle:
    config = pickle.load(handle)

model = load_model(config.model_path)
y_true, y_pred, fn_prob = build_predictions('test')
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)


Extracting features from audio


  0%|          | 0/217 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step


  0%|          | 1/217 [00:02<10:12,  2.83s/it]

1/1 [==============================] - 0s 23ms/step


...

 99%|█████████▉| 215/217 [05:11<00:03,  1.74s/it]

1/1 [==============================] - 0s 19ms/step


100%|█████████▉| 216/217 [05:12<00:01,  1.55s/it]

1/1 [==============================] - 0s 30ms/step


100%|██████████| 217/217 [05:14<00:00,  1.45s/it]


In [42]:
precision = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
recall = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')
f1 = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

In [43]:
y_probs = []
for i, row in df.iterrows():
    y_prob = fn_prob[row.fname]
    y_probs.append(y_prob)
    for c,p in zip(classes, y_prob):
        df.at[i,c] = p

In [44]:
y_pred = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_pred


In [45]:
df.to_csv('predictions.csv')

In [46]:
print("Accuracy:", acc_score)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.6320509794666037
Precision: 0.6404258571945903
Recall: 0.6320509794666037
F1 Score: 0.6338740852124337
